# Homework 4
# Authors: Ashutosh Ekade, Zechen Liu, Jessica Ghai, Yuetong Li

In [1]:
import pandas as pd
import numpy as np

In [2]:
interest_rates = pd.read_excel('~/Downloads/Homework_4_Data_Paths.xlsx', skiprows=4)
interest_rates.columns = [i for i in range(11)]
print(interest_rates.shape)
average_interest_rates = interest_rates.mean(axis=1)
print(interest_rates.head())

(200, 11)
     0         1         2         3         4         5         6         7   \
0  0.03  0.034538  0.036521  0.032033  0.032885  0.030994  0.033656  0.042158   
1  0.03  0.044079  0.040815  0.034202  0.030071  0.023085  0.023245  0.041680   
2  0.03  0.023008  0.016484  0.026603  0.045279  0.043511  0.057754  0.049811   
3  0.03  0.032062  0.031301  0.027659  0.024445  0.034447  0.046123  0.045076   
4  0.03  0.028467  0.040712  0.038105  0.037651  0.029367  0.028553  0.041553   

         8         9         10  
0  0.049124  0.057050  0.048267  
1  0.036565  0.044381  0.034728  
2  0.041742  0.037444  0.045461  
3  0.036885  0.039181  0.040358  
4  0.033604  0.048510  0.051054  


Q1. Discount Factors

In [17]:
maturities = range(1, 11)
discount_factors_df = pd.DataFrame(index=range(len(interest_rates)), columns=maturities)

for index, row in interest_rates.iterrows():
    for maturity in maturities:
        avg_rate = np.mean(row[:maturity + 1])
        discount_factor = np.exp(-(maturity*avg_rate)) # Discount factor using the average rate
        discount_factors_df.at[index, maturity] = discount_factor

# Printing the calculated discount factors
print("Discount Factors Dataframe: \n", discount_factors_df.head(), "\n")
print("Prices of Zero coupon bonds with maturities from 1 to 10 years: \n", discount_factors_df.mean(axis=0))

Discount Factors Dataframe: 
          1         2         3         4         5         6         7   \
0  0.968246  0.934847  0.905001  0.875656  0.848621  0.820632  0.787661   
1  0.963638  0.926264  0.894204  0.866466  0.844895  0.824249  0.791537   
2  0.973844  0.954729  0.930465  0.893062  0.857211  0.812226  0.774224   
3  0.969446  0.939656  0.913231  0.890143   0.86077  0.823867  0.788815   
4   0.97119   0.93602  0.902161  0.869403  0.843452  0.819066  0.786543   

         8         9         10  
0  0.751156  0.711014   0.67815  
1  0.763385  0.731023  0.706067  
2     0.743  0.715715  0.684422  
3  0.760503  0.731645  0.703065  
4  0.760492  0.725517  0.690372   

Prices of Zero coupon bonds with maturities from 1 to 10 years: 
 1     0.968848
2     0.936725
3     0.904764
4     0.873446
5        0.843
6     0.813496
7     0.784974
8     0.757398
9     0.730767
10    0.705043
dtype: object


Q2. Pricing of interest-rate cap for strike-rate of 0.037

In [5]:
path_index = 0
num_paths = len(interest_rates)  # Total number of paths

strike_rate = 0.037
notional_value = 100

# Across all paths we will discount the cash flows using the average interest rate, then compute the price of the option
option_prices_all_paths = []
for path_index in range(num_paths):
    cash_flows = []
    for maturity in range(1, 6):
        short_rate = interest_rates.iloc[path_index, maturity]
        cash_flow = max(0, short_rate - strike_rate) * notional_value
        cash_flows.append(cash_flow)

    discounted_cash_flows = []
    for i, cash_flow in enumerate(cash_flows):
        avg_short_rate = interest_rates.iloc[path_index, :i + 1].mean()
        discount_factor = 1 / (1 + avg_short_rate) ** (i + 1)
        discounted_cash_flow = cash_flow * discount_factor
        discounted_cash_flows.append(discounted_cash_flow)

    option_price = sum(discounted_cash_flows)
    option_prices_all_paths.append(option_price)

average_option_price = sum(option_prices_all_paths) / num_paths
print("Price of a 5-year interest rate cap with strike-rate {} is: $ {:.5f}".format(strike_rate, average_option_price))

Price of a 5-year interest rate cap with strike-rate 0.037 is: $ 0.99627


Q3. Pricing of interest-rate floor for strike-rate of 0.02

In [7]:
path_index = 0
num_paths = len(interest_rates)

strike_rate = 0.020
notional_value = 100

# Repeat the same previous exercise for the interest-rate floor over all paths
floor_prices_all_paths = []
for path_index in range(num_paths):
    cash_flows = []
    for maturity in range(1, 6):
        short_rate = interest_rates.iloc[path_index, maturity]
        cash_flow = max(0, strike_rate - short_rate) * notional_value
        cash_flows.append(cash_flow)

    discounted_cash_flows = []
    for i, cash_flow in enumerate(cash_flows):
        avg_short_rate = interest_rates.iloc[path_index, :i + 1].mean()
        discount_factor = 1 / (1 + avg_short_rate) ** (i + 1)
        discounted_cash_flow = cash_flow * discount_factor
        discounted_cash_flows.append(discounted_cash_flow)

    floor_price = sum(discounted_cash_flows)
    floor_prices_all_paths.append(floor_price)

average_floor_price = sum(floor_prices_all_paths) / num_paths
print("Price of a 5-year interest rate floor with strike-rate {} is: $ {:.5f}".format(strike_rate, average_floor_price))

Price of a 5-year interest rate floor with strike-rate 0.02 is: $ 0.02715


Q4. Pricing of a 10-year range accrual note with LB & UB at 2& % 4%

In [8]:
lower_bound = 0.02
upper_bound = 0.04

notional_value = 100

# DataFrame to store the cash flows for each path
cash_flows_df = pd.DataFrame(index=range(len(interest_rates)), columns=range(1, 11))

# Computing cashflows
for path_index in range(len(interest_rates)):
    for year in range(1, 11):  # 10-year note
        accrued_coupon = 0
        short_rate = interest_rates.iloc[path_index, year]
        if lower_bound <= short_rate <= upper_bound:
            accrued_coupon = short_rate * notional_value
        cash_flows_df.at[path_index, year] = accrued_coupon

# Dataframe to store PV of cashflows
discounted_cash_flows_df = pd.DataFrame(index=range(len(interest_rates)), columns=range(1, 11))

# Discounting the CFs
for path_index in range(len(interest_rates)):
    for maturity in range(1, 11):  # 10-year note
        accrued_coupon = cash_flows_df.at[path_index, maturity]
        discounted_cash_flow = accrued_coupon * discount_factors_df.at[path_index, maturity]
        discounted_cash_flows_df.at[path_index, maturity] = discounted_cash_flow

price_range_accrual_note = discounted_cash_flows_df.mean().sum()
print("PV of the Cashflows for first 5 paths on notional value of $ 100: \n")
print(discounted_cash_flows_df.head(), "\n")
# Print the price of the range accrual note
print("Price of a 10-year range accrual note with lower & upper bounds at 2% & 4% resp. is: $ {:.5f}".format(price_range_accrual_note))

PV of the Cashflows for first 5 paths on notional value of $ 100: 

         1         2         3         4         5         6    7         8   \
0   3.34414  3.414119  2.899008  2.879585  2.630189  2.761933  0.0       0.0   
1       0.0       0.0  3.058344  2.605507  1.950477  1.915991  0.0  2.791352   
2  2.240588       0.0  2.475355       0.0       0.0       0.0  0.0       0.0   
3  3.108227  2.941203  2.525865  2.175976  2.965124       0.0  0.0  2.805089   
4  2.764666       0.0  3.437719   3.27342  2.476984  2.338706  0.0  2.555594   

         9         10  
0       0.0       0.0  
1       0.0  2.452003  
2  2.679944       0.0  
3  2.866652       0.0  
4       0.0       0.0   

Price of a 10-year range accrual note with lower & upper bounds at 2% & 4% resp. is: $ 18.49890


Q5. Pricing of a 5-year inverse floater

In [9]:
notional_value = 100

# DataFrame to store the cash flows for each path
cash_flows_df = pd.DataFrame(index=range(len(interest_rates)), columns=range(1, 6))

# Computing cashflows
for path_index in range(len(interest_rates)):
    for year in range(1, 6):  # Five-year inverse floater
        short_rate = interest_rates.iloc[path_index, year]
        cash_flow = (2 * short_rate - 0.06) * notional_value
        cash_flows_df.at[path_index, year] = cash_flow

# Dataframe to store PV of cashflows
discounted_cash_flows_df = pd.DataFrame(index=range(len(interest_rates)), columns=range(1, 6))

# Discounting the CFs
for path_index in range(len(interest_rates)):
    for year in range(1, 6):  # Five-year inverse floater
        accrued_coupon = cash_flows_df.at[path_index, year]
        discounted_cash_flow = accrued_coupon * discount_factors_df.at[path_index, year]
        discounted_cash_flows_df.at[path_index, year] = discounted_cash_flow

price_inverse_floater = discounted_cash_flows_df.mean().sum()
print("PV of the Cashflows for first 5 paths on notional value of $ 100: \n")
print(discounted_cash_flows_df.head(), "\n")
print("Price of the 5-year inverse floater: $ {:.5f}".format(price_inverse_floater))

PV of the Cashflows for first 5 paths on notional value of $ 100: 

          1         2         3         4         5
0  0.878805  1.219156  0.368008   0.50523  0.168649
1  2.713336  2.003473  0.751465  0.012219 -1.168417
2 -1.361889 -2.580829 -0.632077  2.729038  2.316366
3  0.399782  0.244472 -0.427659 -0.988906   0.76563
4 -0.297807  2.005241  1.462474  1.330419 -0.106746 

Price of the 5-year inverse floater: $ 4.46483


Q6. Comparing Prices with different sets of interest rates

In [10]:
# We will use the code from Q2 to compute the Price of EU Call option on strike 0.04

path_index = 0
num_paths = len(interest_rates)  # Total number of paths

strike_rate = 0.04
notional_value = 100

# Across all paths we will discount the cash flows using the average interest rate, then compute the price of the option
option_prices_all_paths = []
for path_index in range(num_paths):
    cash_flows = []
    for maturity in range(1, 6):
        short_rate = interest_rates.iloc[path_index, maturity]
        cash_flow = max(0, short_rate - strike_rate) * notional_value
        cash_flows.append(cash_flow)

    discounted_cash_flows = []
    for i, cash_flow in enumerate(cash_flows):
        avg_short_rate = interest_rates.iloc[path_index, :i + 1].mean()
        discount_factor = 1 / (1 + avg_short_rate) ** (i + 1)
        discounted_cash_flow = cash_flow * discount_factor
        discounted_cash_flows.append(discounted_cash_flow)

    option_price = sum(discounted_cash_flows)
    option_prices_all_paths.append(option_price)

average_option_price = sum(option_prices_all_paths) / num_paths
print("Price of a 5-year interest rate cap on short-rate with strike-rate {} is: $ {:.5f}".format(strike_rate, average_option_price))

Price of a 5-year interest rate cap on short-rate with strike-rate 0.04 is: $ 0.55103


In [11]:
five_year_average = interest_rates.iloc[:,:5].mean(axis=1).to_list()
path_index = 0
num_paths = len(interest_rates)  # Total number of paths

strike_rate = 0.04
notional_value = 100

# Across all paths we will discount the cash flows using the average interest rate, then compute the price of the option
option_prices_all_paths = []
for path_index in range(num_paths):
    cash_flows = []
    for maturity in range(1, 6):
        short_rate = five_year_average[path_index]
        cash_flow = max(0, short_rate - strike_rate) * notional_value
        cash_flows.append(cash_flow)

    discounted_cash_flows = []
    for i, cash_flow in enumerate(cash_flows):
        avg_short_rate = interest_rates.iloc[path_index, :i + 1].mean()
        discount_factor = 1 / (1 + avg_short_rate) ** (i + 1)
        discounted_cash_flow = cash_flow * discount_factor
        discounted_cash_flows.append(discounted_cash_flow)

    option_price = sum(discounted_cash_flows)
    option_prices_all_paths.append(option_price)

average_option_price = sum(option_prices_all_paths) / num_paths
print("Price of a 5-year interest rate cap on average-rate with strike-rate {} is: $ {:.5f}".format(strike_rate, average_option_price))

Price of a 5-year interest rate cap on average-rate with strike-rate 0.04 is: $ 0.06622


Here we can see that the 5-year European call option (caplet) on the short-term interest rate is more valuable ($0.55) that the five-year European call option on the average short-term interest rate during the five years ($0.06).